# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import signal
import logging
import subprocess
import time
from gunicorn.glogging import Logger as GLogger

In [ ]:
#| export
class GunicornLogger(GLogger):
    def setup(self, cfg):
        super(GunicornLogger, self).setup(cfg)
        self.access_log = logging
        self.error_log = logging

class TerminateProtected:
    kill_now = False

    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
        signal.signal(signal.SIGTSTP, self.exit_gracefully)

    def exit_gracefully(self, *args):
        logging.warning("Received SIGINT but executing anyway.")
        self.kill_now = True

In [ ]:
#| eval: false
worker_cmd = ['gunicorn', '-b', '0.0.0.0:5009', '-w', '5', 'worker:app', '-n', 'worker', '--access-logfile', '-']
model_cmd = ['gunicorn', '-b', '0.0.0.0:5001', '-w', '1', 'model:app', '-n', 'model', '--access-logfile', '-']
worker = subprocess.Popen(worker_cmd)
model = subprocess.Popen(model_cmd)
graceful_killer = TerminateProtected()
while not graceful_killer.kill_now:
    time.sleep(1)
    if worker.poll() is None or model.poll() is None: continue
    else:
        worker.send_signal(15)
        model.send_signal(15)
